In [1]:
import credentials
import settings

import re
import tweepy
import mysql.connector
import pandas as pd
from textblob import TextBlob

class MyStreamerListener(tweepy.StreamListener):
    def on_status(self, status):
        if status.retweeted:
            return False
        id_str = status.id_str
        created_at = status.created_at
        text = deEmojify(status.text)
        sentiment = TextBlob(status.text)
        polarity = sentiment.polarity
        subjectivity = sentiment.subjectivity
        user_created_at = status.user.created_at
        user_location = deEmojify(status.user.location)
        user_description = deEmojify(status.user.description)
        user_followers_count = status.user.followers_count
        longitude = None
        latitude = None
        if status.coordinates:
            longitude= status.coordinates['coordinates']
            latitude = status.coordinates['coordinates']
        retweet_count = status.retweet_count
        favorite_count = status.favorite_count
        print(status.text)
        print("Long: {}, Lat: {}". format(longitude, latitude))
        
        if mydb.is_connected():
            mycursor = mydb.cursor()
            sql = "INSERT INTO {} (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(settings.TABLE_NAME)
            val = (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location,user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count)
            mycursor.execute(sql,val)
            mydb.commit()
            mycursor.close()


In [2]:
def clean_tweet(self, tweet): 

    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) \
                                |(\w+:\/\/\S+)", " ", tweet).split()) 
                                
def deEmojify(text):
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

In [3]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="gaurav",
    database="TwitterDB",
    charset = 'utf8'
)
if mydb.is_connected():
    mycursor = mydb.cursor()
    mycursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(settings.TABLE_NAME))
    if mycursor.fetchone()[0] != 1:
        mycursor.execute("CREATE TABLE {} ({})".format(settings.TABLE_NAME, settings.TABLE_ATTRIBUTES))
        mydb.commit()
    mycursor.close()


In [4]:
auth  = tweepy.OAuthHandler(credentials.CONSUMER_KEY, credentials.CONSUMER_SECRET)
auth.set_access_token(credentials.ACCESS_TOKEN, credentials.ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [5]:
myStreamerListener = MyStreamerListener()
myStream = tweepy.Stream(auth = api.auth, listener = myStreamerListener)
myStream.filter(languages=["en"], track = settings.TRACK_WORDS)
mydb.close()

T @TheYanipopo: Dont ever compare James to Jeffree again. #JeffreeStarApproved https://t.co/YI8bZcQg0a
Long: None, Lat: None
RT @JeffreeStar: Just sent you $250 💖
Long: None, Lat: None
@JeffreeStar #JeffreeStarApproved You are doing such amazing things for people.. lost my job due to what has been g… https://t.co/niD9217vtS
Long: None, Lat: None
RT @cxtyofstars: i have a spinal injury and have had to move in with my parents the last year as I cannot put on my own shoes. my parents a…
Long: None, Lat: None
@JeffreeStar $staples0195 #JeffreeStarApproved
Long: None, Lat: None
RT @JeffreeStar: It's the least I can do! Hope this helps, check your account! xo #JeffreeStarApproved
Long: None, Lat: None
RT @JeffreeStar: I gothchu! Hope this helps with rent!
Long: None, Lat: None
@JeffreeStar Just trying to pay off my debt so I can start saving to buy my boyfriend and I a home so we can get of… https://t.co/B9wz2trYO6
Long: None, Lat: None
@JeffreeStar #JeffreeStarApproved I'm really trying to 

ReadTimeoutError: HTTPSConnectionPool(host='stream.twitter.com', port=443): Read timed out.